Este notebook sirve para entrenar un modelo **LLM** basado en Mistral para clasificar atacantes en análisis de tráfico de red de forma guiada. Para usar el modelo es necesario un entorno que cuente con una GPU potente (se ha probado con una GPU T4)

Si se usa en la plataforma Google Colab, es útil montar Google Drive y crear un directorio en el que trabajar, en todo caso deben de consultarse las rutas utilizadas en este ejemplo y adaptarse.

Se utiliza la herramienta axolotl para conseguir entrenar el modelo y ajustar los parámetros de forma más cómoda.

El notebook se basa en el trabajo de Dhruti Davey y su repositorio de github https://github.com/dhd2000/DissertationFiles


El resultado es un directorio "qlora.out" que contiene el modelo, el cuál se puede probar utilizando el notebook "Prueba_modelo.ipynb"


*Ivan Dunlap, 2024*

## PASO 1: instalación de dependencias necesarias en el entorno
Para ejecutar este notebook, es necesario instalar varias dependencias. A continuación, se muestra el código para instalar estas dependencias en el entorno de ejecución.

In [ ]:
%cd /content/drive/MyDrive/PASIR/model
!pip install torch torchvision torchaudio
!pip install datasets
!pip install transformers
!pip install aggregate
!pip install accelerate
!pip install flash_attn
!pip install argparse
!pip install -e git+https://github.com/OpenAccess-AI-Collective/axolotl#egg=axolotl
!pip install huggingface_hub==0.23.3

## PASO 2: autenticación en Hugging Face (ver sección obtención de token)

Para utilizar modelos de la plataforma Hugging Face, es necesario obtener un token y logearse.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## PASO 3: clonar los repositorios necesarios
Esta serie de repositorios es necesaria para crear el modelo.

In [ ]:
!git clone https://github.com/dhd2000/DissertationFiles.git
!git clone https://github.com/KayvanKarim/ntfa.git
!git clone https://github.com/OpenAccess-AI-Collective/axolotl.git
!git clone https://github.com/DunlapIvan-Clase/PASIR.git

## PASO 4: descarga y extracción del dataset CIDDS

Para hacer fine-tuning a nuestro modelo base (Mistral) vamos a utilizar un dataset con tráfico previamente etiquetado, como el Intrusion detection evaluation dataset (CIC-IDS2017), en este paso se descarga y se extrae, en caso de ya contar con el o usar otro similar, omitir este paso.

El dataset es un fichero csv con el tráfico etiquetado de la siguiente forma:



```
2017-03-24 12:52:33.504,    0.000,TCP  ,10998_36,   443,192.168.220.16, 40858,       1,      66,    1,.A...., 32,normal,---,---,---
2017-03-24 12:52:32.711,    0.000,TCP  ,10056_186,   443,192.168.220.4, 37516,       1,      66,    1,.A...., 32,normal,---,---,---
2017-03-24 12:52:32.617,    0.000,ICMP ,192.168.220.1,     0,192.168.220.15,   3.1,       1,      70,    1,......,192,victim,pingScan,52,nmap args: -n -sP -PE -T 1
```



In [ ]:
!wget https://www.hs-coburg.de/fileadmin/hscoburg/WISENT-CIDDS-001.zip -O WISENT-CIDDS-001.zip
!unzip WISENT-CIDDS-001.zip -d ./CIDDS-001

## PASO 5: procesar los datos para entrenar el modelo

Una vez tengamos nuestro dataset, debemos procesarlo para que nuestro modelo lo entienda, para ello nos valemos de una serie de scripts en cadena.

A la hora de ejecutar el script ntfa, si no disponemos de un fichero de configuración json, debemos de configurar el índice de cada columna de nuestro dataset para que el script la reconozca.

El resultado de la ejecución de esta celda, debe crear un fichero "output.jsonl" que tendrá un contenido similar a este:



```
{"instruction": "Check if the given traffic flow is normal or of an attacker or a victim", "input": "Start Frame: 2017-03-15T00:01:16.632000, End Frame: 2017-03-15T00:02:16.632000, Source IP: 192.168.100.5, Duration: 0.0, Protocol: 1, Source Port: 1, Destination IP: 14, Destination Port: 14, Number of Packets: 17", "output": "normal"}
{"instruction": "Check if the given traffic flow is normal or of an attacker or a victim", "input": "Start Frame: 2017-03-15T00:01:16.632000, End Frame: 2017-03-15T00:02:16.632000, Source IP: 192.168.220.15, Duration: 0.047, Protocol: 1, Source Port: 2, Destination IP: 2, Destination Port: 2, Number of Packets: 6", "output": "attacker"}
{"instruction": "Check if the given traffic flow is normal or of an attacker or a victim", "input": "Start Frame: 2017-03-15T00:01:16.632000, End Frame: 2017-03-15T00:02:16.632000, Source IP: 192.168.220.16, Duration: 2.303, Protocol: 2, Source Port: 48, Destination IP: 14, Destination Port: 12, Number of Packets: 179", "output": "normal"}
```



In [ ]:
import os

# Directorio donde se encuentra el archivo CSV
traffic_dir = r'./CIDDS-001/CIDDS-001/traffic/OpenStack'

# Nombre del archivo específico a utilizar
target_csv_file = 'CIDDS-001-internal-week1.csv'

target_csv_path = os.path.join(traffic_dir, target_csv_file)

!python3 ntfa/ntfa.py {target_csv_path}
!python3 DissertationFiles/convert.py output.csv
!python3 DissertationFiles/balance_dataset.py output.jsonl

## PASO 6: entrenamiento del modelo

Cuando se cree el fichero "output.jsonl" ya es posible entrenar el modelo, en caso de querer realizar algún cambio en los parametros del fine-tuning es necesario editar el fichero "config.yaml"

El resultado es el modelo en el directorio "qlora.out"

In [ ]:
!mkdir mistral-NIDS/
%cd mistral-NIDS/
!cp ../DissertationFiles/config/mistral_config.yaml ./config.yaml
!sed -i "s/flash_attention: true/flash_attention: false/" ./config.yaml # Desactivar flash_attention
!accelerate launch -m axolotl.cli.train config.yaml